In [2]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from torchvision.utils import save_image
from visualdl import LogWriter

In [3]:
class GNet(nn.Module):
    def __init__(self, latent_size=64, n_channel=3, n_g_feature=64):
        super().__init__()
        self.layer = nn.Sequential(
            # 第1层：从潜在矢量上采样到4 * n_g_feature特征图
            nn.ConvTranspose2d(latent_size, 4 * n_g_feature, kernel_size=4, bias=False),
            nn.BatchNorm2d(4 * n_g_feature),
            nn.ReLU(),

            # 第2层：上采样到2 * n_g_feature特征图
            nn.ConvTranspose2d(4 * n_g_feature, 2 * n_g_feature, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(2 * n_g_feature),
            nn.ReLU(),

            # 第3层：上采样到n_g_feature特征图
            nn.ConvTranspose2d(2 * n_g_feature, n_g_feature, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(n_g_feature),
            nn.ReLU(),

            # 第4层：上采样到n_channel特征图（最终输出）
            nn.ConvTranspose2d(n_g_feature, n_channel, kernel_size=4, stride=2, padding=1),
            
            # 输出激活函数
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.layer(x)
        return x

In [4]:
class DNet(nn.Module):
    def __init__(self, n_d_feature=64, n_channel=3):
        super().__init__()
        self.layer = nn.Sequential(
            # 第1层：卷积操作，将输入特征图从n_channel降采样到n_d_feature
            nn.Conv2d(n_channel, n_d_feature, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),

            # 第2层：卷积操作，将特征图从n_d_feature升采样到2 * n_d_feature
            nn.Conv2d(n_d_feature, 2 * n_d_feature, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(2 * n_d_feature),
            nn.LeakyReLU(0.2),

            # 第3层：卷积操作，将特征图从2 * n_d_feature升采样到4 * n_d_feature
            nn.Conv2d(2 * n_d_feature, 4 * n_d_feature, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(4 * n_d_feature),
            nn.LeakyReLU(0.2),

            # 第4层：卷积操作，将特征图从4 * n_d_feature降采样到1（最终输出）
            nn.Conv2d(4 * n_d_feature, 1, kernel_size=4)
        )

    def forward(self, x):
        x = self.layer(x)
        return x

In [5]:
def weights_init(m):
    if type(m) in [nn.ConvTranspose2d, nn.Conv2d]:
        init.xavier_normal_(m.weight)
    elif type(m) == nn.BatchNorm2d:
        init.normal_(m.weight, 1.0, 0.02)
        init.constant_(m.bias, 0)



In [6]:
latent_size = 64  # 潜在向量的大小，即生成器的输入大小
n_channel = 3  # 输入图像的通道数（例如，RGB图像的通道数为3）
n_g_feature = 64  # 生成器中间层的特征数

# 创建生成器和鉴别器网络的实例
gnet = GNet()
dnet = DNet()

# 使用自定义的权重初始化函数初始化网络的权重
gnet.apply(weights_init)
dnet.apply(weights_init)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

gnet.to(device)
dnet.to(device)

# 下载CIFAR-10数据集
dataset = CIFAR10(root='./CIFARdata', download=True, transform=transforms.ToTensor())

# 创建数据加载器以处理CIFAR-10数据集的批次
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# 定义带Logits的二元交叉熵损失函数
criterion = nn.BCEWithLogitsLoss()

# 为生成器和鉴别器网络创建Adam优化器
goptimizer = torch.optim.Adam(gnet.parameters(), lr=0.0002, betas=(0.5, 0.999))
doptimizer = torch.optim.Adam(dnet.parameters(), lr=0.0002, betas=(0.5, 0.999))

# 设置训练的批次大小
batch_size = 64

# 生成用于在训练期间可视化生成器进展的固定随机噪声
fixed_noises = torch.randn(batch_size, latent_size, 1, 1)

Files already downloaded and verified


In [7]:
epoch_num = 16  # 训练的总轮数
dlosses = []  # 用于保存鉴别器损失的列表
glosses = []  # 用于保存生成器损失的列表

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  
for epoch in range(epoch_num):  # 遍历
    for batch_idx, data in enumerate(dataloader):  # 遍历数据加载器中的每个批次
        real_images, _ = data  # 从数据中获取真实图像和标签，这里的标签在训练中未被使用
        real_images = real_images.to(device)  # 将真实图像移动到指定的设备
        batch_size = real_images.size(0)  # 获取批次大小

        # 针对真实图像进行鉴别器的训练
        labels = torch.ones(batch_size)  # 创建用于真实图像的标签
        preds = dnet(real_images)  # 鉴别器对真实图像的预测
        outputs = preds.reshape(-1)  # 将预测结果展平
        dloss_real = criterion(outputs, labels)  # 计算鉴别器对真实图像的损失
        dmean_real = outputs.sigmoid().mean()  # 计算鉴别器对真实图像的平均输出（用于监控）

        # 针对生成的假图像进行鉴别器的训练
        noises = torch.randn(batch_size, latent_size, 1, 1)  # 生成随机噪声
        fake_images = gnet(noises)  # 生成器生成假图像
        labels = torch.zeros(batch_size)  # 创建用于假图像的标签
        fake = fake_images.detach()  # 将生成的假图像从生成器的计算图中分离，用于避免梯度传播到生成器
        preds = dnet(fake)  # 鉴别器对假图像的预测
        outputs = preds.view(-1)  # 将预测结果展平
        dloss_fake = criterion(outputs, labels)  # 计算鉴别器对假图像的损失
        dmean_fake = outputs.sigmoid().mean()  # 计算鉴别器对假图像的平均输出（用于监控）

        dloss = dloss_real + dloss_fake  # 计算总的鉴别器损失
        dnet.zero_grad()  # 清零鉴别器的梯度
        dloss.backward()  # 反向传播计算梯度
        doptimizer.step()  # 更新鉴别器的参数

        # 针对生成器进行训练
        labels = torch.ones(batch_size)  # 创建用于生成器的标签
        preds = dnet(fake_images)  # 鉴别器对生成器生成的假图像的预测
        outputs = preds.view(-1)  # 将预测结果展平
        gloss = criterion(outputs, labels)  # 计算生成器的损失
        gmean_fake = outputs.sigmoid().mean()  # 计算生成器生成的假图像的平均输出（用于监控）
        gnet.zero_grad()  # 清零生成器的梯度
        gloss.backward()  # 反向传播计算梯度
        goptimizer.step()  # 更新生成器的参数

        # 将损失记录到列表中，用于后续的可视化或分析
        dlosses.append(dloss)
        glosses.append(gloss)

        if batch_idx % 100 == 0:
            # 生成一些固定噪声上的图像，并保存用于可视化生成器的进展
            fake = gnet(fixed_noises)
            save_image(fake, f'./GAN_saved/images_epoch{epoch:02d}_batch{batch_idx:03d}.png')

            # 打印训练进度和损失
            print(f'Epoch index: {epoch}, {epoch_num} epoches in total.')
            print(f'Batch index: {batch_idx}, the batch size is {batch_size}.')
            print(f'Discriminator loss is: {dloss}, generator loss is: {gloss}', '\n',
                  f'Discriminator tells real images real ability: {dmean_real}', '\n',
                  f'Discriminator tells fake images real ability: {dmean_fake:g}/{gmean_fake:g}')

# 使用tensorboard进行训练过程中损失的可视化
with LogWriter(logdir="./log/dloss") as writer:
    for step in range(len(dlosses)):
        writer.add_scalar(tag="loss", step=step, value=float('%.4f' % (dlosses[step])))

with LogWriter(logdir="./log/gloss") as writer:
    for step in range(len(glosses)):
        writer.add_scalar(tag="loss", step=step, value=float('%.4f' % (glosses[step])))


gnet_save_path = 'gnet.pt'
torch.save(gnet, gnet_save_path)
dnet_save_path = 'dnet.pt'
torch.save(dnet, dnet_save_path)

for i in range(100):
    noises = torch.randn(batch_size, latent_size, 1, 1)
    fake_images = gnet(noises)
    save_image(fake_images, f'./test_GAN/{i}.png')
print(gnet, dnet)

Epoch index: 0, 16 epoches in total.
Batch index: 0, the batch size is 64.
Discriminator loss is: 1.8710863590240479, generator loss is: 3.898284912109375 
 Discriminator tells real images real ability: 0.5502840280532837 
 Discriminator tells fake images real ability: 0.687547/0.0223098
Epoch index: 0, 16 epoches in total.
Batch index: 100, the batch size is 64.
Discriminator loss is: 0.6950627565383911, generator loss is: 2.5055689811706543 
 Discriminator tells real images real ability: 0.5719171166419983 
 Discriminator tells fake images real ability: 0.00479252/0.0976063
Epoch index: 0, 16 epoches in total.
Batch index: 200, the batch size is 64.
Discriminator loss is: 0.6278464794158936, generator loss is: 5.240541934967041 
 Discriminator tells real images real ability: 0.9103367924690247 
 Discriminator tells fake images real ability: 0.374452/0.00753936
Epoch index: 0, 16 epoches in total.
Batch index: 300, the batch size is 64.
Discriminator loss is: 0.37266096472740173, gene

Epoch index: 3, 16 epoches in total.
Batch index: 500, the batch size is 64.
Discriminator loss is: 1.1413798332214355, generator loss is: 2.734492301940918 
 Discriminator tells real images real ability: 0.882859468460083 
 Discriminator tells fake images real ability: 0.613689/0.0755354
Epoch index: 3, 16 epoches in total.
Batch index: 600, the batch size is 64.
Discriminator loss is: 1.092395544052124, generator loss is: 0.8793870806694031 
 Discriminator tells real images real ability: 0.4148605763912201 
 Discriminator tells fake images real ability: 0.111898/0.448631
Epoch index: 3, 16 epoches in total.
Batch index: 700, the batch size is 64.
Discriminator loss is: 1.2005987167358398, generator loss is: 2.3576889038085938 
 Discriminator tells real images real ability: 0.8363547325134277 
 Discriminator tells fake images real ability: 0.602977/0.113339
Epoch index: 4, 16 epoches in total.
Batch index: 0, the batch size is 64.
Discriminator loss is: 0.749626100063324, generator lo

Epoch index: 7, 16 epoches in total.
Batch index: 200, the batch size is 64.
Discriminator loss is: 0.8453316688537598, generator loss is: 1.1271940469741821 
 Discriminator tells real images real ability: 0.546355664730072 
 Discriminator tells fake images real ability: 0.170392/0.371726
Epoch index: 7, 16 epoches in total.
Batch index: 300, the batch size is 64.
Discriminator loss is: 0.8760287165641785, generator loss is: 1.4081542491912842 
 Discriminator tells real images real ability: 0.6035906076431274 
 Discriminator tells fake images real ability: 0.257233/0.276963
Epoch index: 7, 16 epoches in total.
Batch index: 400, the batch size is 64.
Discriminator loss is: 0.6787316203117371, generator loss is: 1.9618333578109741 
 Discriminator tells real images real ability: 0.7395293712615967 
 Discriminator tells fake images real ability: 0.272485/0.164046
Epoch index: 7, 16 epoches in total.
Batch index: 500, the batch size is 64.
Discriminator loss is: 0.8724157810211182, generato

Epoch index: 10, 16 epoches in total.
Batch index: 700, the batch size is 64.
Discriminator loss is: 1.180700421333313, generator loss is: 1.17445969581604 
 Discriminator tells real images real ability: 0.371432363986969 
 Discriminator tells fake images real ability: 0.0632257/0.342642
Epoch index: 11, 16 epoches in total.
Batch index: 0, the batch size is 64.
Discriminator loss is: 1.344381332397461, generator loss is: 1.021256446838379 
 Discriminator tells real images real ability: 0.3901611268520355 
 Discriminator tells fake images real ability: 0.197845/0.409138
Epoch index: 11, 16 epoches in total.
Batch index: 100, the batch size is 64.
Discriminator loss is: 0.8309995532035828, generator loss is: 1.2563976049423218 
 Discriminator tells real images real ability: 0.5325959920883179 
 Discriminator tells fake images real ability: 0.126106/0.312947
Epoch index: 11, 16 epoches in total.
Batch index: 200, the batch size is 64.
Discriminator loss is: 0.8715070486068726, generator 

Epoch index: 14, 16 epoches in total.
Batch index: 400, the batch size is 64.
Discriminator loss is: 1.1444573402404785, generator loss is: 2.4048352241516113 
 Discriminator tells real images real ability: 0.8101243376731873 
 Discriminator tells fake images real ability: 0.536992/0.113789
Epoch index: 14, 16 epoches in total.
Batch index: 500, the batch size is 64.
Discriminator loss is: 0.7592909932136536, generator loss is: 2.2633438110351562 
 Discriminator tells real images real ability: 0.7861499190330505 
 Discriminator tells fake images real ability: 0.353558/0.123772
Epoch index: 14, 16 epoches in total.
Batch index: 600, the batch size is 64.
Discriminator loss is: 0.5070636868476868, generator loss is: 2.4792423248291016 
 Discriminator tells real images real ability: 0.7214469313621521 
 Discriminator tells fake images real ability: 0.140863/0.109906
Epoch index: 14, 16 epoches in total.
Batch index: 700, the batch size is 64.
Discriminator loss is: 0.7485439777374268, gen